<a href="https://colab.research.google.com/github/Sreekar-Kandhadai/pyspark-interview-questions/blob/main/Imp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
𝐐𝐮𝐞𝐬𝐭𝐢𝐨𝐧:
You have two datasets with different keys, and you need to join them based on a specific mapping logic. Here are the datasets:

Task: Write a PySpark program to join these datasets such that category_id in Dataset 1 matches with cat_id in Dataset 2, and
produce the result with the following schema: | product_id | product_name | category_name |

𝐬𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐝𝐚𝐭𝐚𝐬𝐞𝐭
# Create Dataset 1 (Products)

products_data = [ (1, "Laptop", 101), (2, "Smartphone", 102), (3, "Tablet", 101) ]

products_schema = ["product_id", "product_name", "category_id"]

products_df = spark.createDataFrame(products_data, schema=products_schema)

# Create Dataset 2 (Categories)

categories_data = [ (101, "Electronics"), (102, "Mobile"), (103, "Home Appliance") ]

categories_schema = ["cat_id", "category_name"]


In [8]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

spark=SparkSession.builder.appName('learning').getOrCreate()

products_data = [ (1, "Laptop", 101), (2, "Smartphone", 102), (3, "Tablet", 101) ]

products_schema = ["product_id", "product_name", "category_id"]

df1=spark.createDataFrame(products_data,products_schema)
df1.show()

categories_data = [ (101, "Electronics"), (102, "Mobile"), (103, "Home Appliance") ]

categories_schema = ["cat_id", "category_name"]

df2=spark.createDataFrame(categories_data,categories_schema)
df2.show()

df3=df1.join(df2,col('category_id')== col('cat_id'),"inner").select("product_id","product_name","category_name")

df3.show()

+----------+------------+-----------+
|product_id|product_name|category_id|
+----------+------------+-----------+
|         1|      Laptop|        101|
|         2|  Smartphone|        102|
|         3|      Tablet|        101|
+----------+------------+-----------+

+------+--------------+
|cat_id| category_name|
+------+--------------+
|   101|   Electronics|
|   102|        Mobile|
|   103|Home Appliance|
+------+--------------+

+----------+------------+-------------+
|product_id|product_name|category_name|
+----------+------------+-------------+
|         1|      Laptop|  Electronics|
|         3|      Tablet|  Electronics|
|         2|  Smartphone|       Mobile|
+----------+------------+-------------+

